In [ ]:
# | hidden: true
# | echo: false
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# some personal style settings to make the plots look nice
# and save some space in the notebook
plt.style.use("../style.mplstyle")